### Ⅰ. 라이브러리

In [61]:
import numpy as np
import pandas as pd

import os
import mido

from mido import MidiFile, MidiTrack, MetaMessage, Message




### Ⅱ. 데이터 추출
- Message
    - type
        - note_on : 음표 시작
        - control_change : 컨트롤러 변경 메시지 ex. 페달 볼륨 조절기 움직임
        - program_change : 프로그램 변경 메시지 ex. 다양한 소리 또는 패치

#### 1. pitch (음정) : [  -8192 to 8192 ]
- 음의 높낮이
- 주파수 표현

#### 2. note (음표) : [  0 to 127 ]
- msg.type = 'note_on' / 'note_off'
    - 소리의 강도 / 세기 ( velocity )
    - 지속 시간 ( time, 단위 : tick )

#### 3. control ( 컨트롤러 번호 ) [ 0 to 127 ]
- https://www.midi.org/specifications/midi-reference-tables/midi-1-0-control-change-messages-data-bytes-2
----
- control=1  ( Modulation Wheel ( 음향 변화 )) ( 0 : 왼쪽, 63 : 중앙 127 : 오른쪽 )
- control=7  ( Main Volume 조절 )
- control=10 ( Pan, 좌우 스테레오 위치 조절 )
- control=64 ( Damper Pedal on/off (Sustain) ) <= 63 Off 64<= On

#### 4. program ( 특정 악기 / 사운드 패치 ) [ 0 to 127 ]
- program_change : 이벤트를 통해, 악기를 변경
- 피아노, 비트 등의 특정한 사운드 지칭
----
- program=0 : 피아노
- program=25 : 악기 관련 사운드
- program=40 : 비트 드럼

In [7]:
file_name = "piano_pirate.mid"
file_path = os.path.join("midi_data", file_name)

mid = MidiFile(file_path)

In [43]:
def distribute_msg(mid_track):
    track_dict = {}
    
    track_dict['MetaMessage'] = []
    track_dict['Message'] = []
    track_dict['ETC'] = []
    
    for msg in mid_track:
        
        if isinstance(msg, MetaMessage):
            track_dict['MetaMessage'].append(msg)
            
        elif isinstance(msg, Message):
            track_dict['Message'].append(msg)
            
        else:
            track_dict['ETC'].append(msg)
            
    return track_dict

In [106]:
def initialize_ticks(mid_track):
    
    df = pd.DataFrame()
    
    time_vec = {}
    time_arr = [0]
    
    for msg in mid_track:
        time = 
        
    time_arr = time_arr[1:]
    
initialize_ticks(track_dict['Message'])

Empty DataFrame
Columns: []
Index: []


In [130]:
len(track_dict['Message'])

1554

#### 4. time // 이것을 어떻게 할 지 좀 더 고민
- tick : mid.ticks_per_beat
    - 1박자의 tick 수를 의미
- tick_interval 을 의미
- time = 0 ( interval = 0 )
    - 이전 이벤트와 동시에 발생
    - 동일한 시간에 기록

In [84]:
mid.tracks[0]

MidiTrack([
  MetaMessage('track_name', name="He's A Pirate", time=0),
  MetaMessage('text', text='By Written by Klaus Badelt', time=0),
  MetaMessage('copyright', text='Arrangement © 2004 Cerullean Pictures ', time=0),
  MetaMessage('copyright', text='Original Score © Walt Disney Pictures', time=0),
  MetaMessage('text', text='Generated by NoteWorthy Composer', time=0),
  MetaMessage('set_tempo', tempo=300000, time=0),
  MetaMessage('key_signature', key='F', time=0),
  MetaMessage('time_signature', numerator=6, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
  MetaMessage('end_of_track', time=0)])

In [83]:
print(mid.ticks_per_beat) # 1박 : print(mid.ticks_per_beat) tick

192


In [99]:
t = 192*30/60
print(t)

96.0


In [ ]:
time_in_seconds = msg.time / (ticks_per_beat * bpm / 60)

In [141]:
for idx, track in enumerate(mid.tracks):
    print(f"Track {idx + 1}: {track.name}")
    
    track_dict = distribute_msg(track)
    
    if len(track_dict['Message'])==0:
        continue;
        
    track_msg = track_dict['Message']
    
    df = pd.DataFrame()
    
    #  msg.type == note_on 에 대해서만
    msg_type = []
    
    msg_note = []
    msg_channel = []
    msg_velocity = []
    msg_interval = []
    
    
    msg_pitch = []
    
    msg_control = []
    msg_wheel = []
    msg_pedal = []
    
    msg_time = [0] # 1분 18초 = 78초
    
    msg_off = []
    msg_con = []
    
    for msg in track_msg:
        
        # note off : note_on and velocity == 0
        if msg.type == 'note_on' and msg.velocity == 0:
            msg_typ = 'note_off'
        else:
            msg_typ = msg.type
            
        msg_type.append(msg_typ)
        
                
        if msg_typ == 'note_on':
            
            time = msg.time + msg_time[-1]
            msg_time.append(time)
            
            #print("NOON", time, msg)
        
            msg_note.append(msg.note)
            msg_channel.append(msg.channel)
            msg_velocity.append(msg.velocity)
            msg_interval.append(msg.time) ## 이전 이벤트와의 간격
        
        elif msg_typ == 'note_off':
            time = msg.time + msg_time[-1]
            msg_off.append(time)
            
        elif msg_typ == 'pitchwheel':
            msg_pitch.append(msg.pitch)
        
        elif msg_typ == 'control_change':
            #print(msg.control, msg.value)
            msg_control.append(msg.control)
            time = msg.time + msg_time[-1]
            msg_con.append(time)
            #print("C", time, msg)
            
        elif msg_typ == 'program_change':
            print("PRPRPRP", time, msg)
        
            
    msg_time = msg_time[1:]
            
    
    print(np.unique(msg_type))
    print(np.unique(msg_control))
    print(msg_time[-1])
    print(len(msg_time))

    print("============================")
    #print(msg_off)
    #print(msg_con)
    print("============================")
            


Track 1: He's A Pirate
Track 2: Right Hand
PRPRPRP 0 program_change channel=0 program=0 time=0
PRPRPRP 0 program_change channel=0 program=0 time=0
['control_change' 'note_off' 'note_on' 'program_change']
[ 7 10 64]
7504
557
Track 3: Left Hand
PRPRPRP 8224 program_change channel=1 program=0 time=0
PRPRPRP 0 program_change channel=1 program=0 time=0
['control_change' 'note_off' 'note_on' 'program_change']
[ 7 10 64]
4372
698


In [230]:
def initialize_note_ticks():
    tick_type = []
    tick_note = []
    tick_chan = []
    tick_velo = []
    
    return tick_type, tick_note, tick_chan, tick_velo

def initialize_ctl_ticks():
    tick_chan = []
    tick_ctl  = []
    tick_val  = []
    
    return tick_chan, tick_ctl, tick_val

def initialize_prog_ticks():
    tick_chan = []
    tick_prog  = []
    
    return tick_chan, tick_prog

In [234]:
for idx, track in enumerate(mid.tracks):
    print(f"Track {idx + 1}: {track.name}")
    
    track_dict = distribute_msg(track)
    
    if len(track_dict['Message'])==0:
        continue;
        
    track_msg = track_dict['Message']
    
    df_note_col = {'tick', 'count', 'msg_type', 'msg_note', 'msg_velocity', 'msg_channel'}
    df_note = pd.DataFrame(columns = df_note_col)
    
    df_control_col = {'tick', 'count','channel', 'control', 'value'}
    df_ctl = pd.DataFrame(columns = df_control_col)
    
    df_program_col = {'tick', 'count', 'channel', 'program'}
    df_prog = pd.DataFrame(columns = df_program_col)
    
    
    note_idx = 0
    note_cur_time = 0
    note_tick_type, note_tick_note, note_tick_chan, note_tick_velo = initialize_note_ticks()
    
    ctl_idx = 0
    ctl_cur_time = 0
    ctl_tick_chan, ctl_tick_ctl, ctl_tick_val = initialize_ctl_ticks()
    
    prog_idx = 0
    prog_cur_time = 0
    prog_tick_chan, prog_tick_prog = initialize_prog_ticks()
    
    
    for msg in track_msg:
        
        # note off : note_on and velocity == 0
        if msg.type == 'note_on' and msg.velocity == 0:
            msg_typ = 'note_off'
        else:
            msg_typ = msg.type
            
        
        if msg.type == 'note_on':
            
            if msg.time > 0:
            
                note_cur_time = msg.time + note_cur_time
                note_idx+=1
            
                df_note.loc[note_idx, "tick"] = note_cur_time
                df_note.loc[note_idx, "count"] = len(note_tick_note)
                df_note.loc[note_idx, "msg_type"] = note_tick_type
                df_note.loc[note_idx, "msg_note"] = note_tick_note
                df_note.loc[note_idx, "msg_velocity"] = note_tick_velo
                df_note.loc[note_idx, "msg_channel"] = note_tick_chan
            
                note_tick_type, note_tick_note, note_tick_chan, note_tick_velo = initialize_note_ticks()
            
            
            note_tick_type.append(msg_typ)
            note_tick_note.append(msg.note)
            note_tick_chan.append(msg.channel)
            note_tick_velo.append(msg.velocity)
            
        if msg.type == 'control_change':
            if msg.time > 0:
                ctl_cur_time = msg.time + ctl_cur_time
                ctl_idx +=1
                
                df_ctl.loc[ctl_idx, "tick"] = ctl_cur_time
                df_ctl.loc[ctl_idx, "count"] = len(ctl_tick_ctl)
                df_ctl.loc[ctl_idx, "channel"] = ctl_tick_chan
                df_ctl.loc[ctl_idx, "control"] = ctl_tick_ctl
                df_ctl.loc[ctl_idx, "value"] = ctl_tick_val
                
                ctl_tick_chan, ctl_tick_ctl, ctl_tick_val = initialize_ctl_ticks()
        
            ctl_tick_chan.append(msg.channel)
            ctl_tick_ctl.append(msg.control)
            ctl_tick_val.append(msg.value)
            
        if msg.type == 'program_change':
            if msg.time >= 0:
                prog_cur_time = msg.time + prog_cur_time
                prog_idx +=1
                
                df_prog.loc[prog_idx, "tick"] = prog_cur_time
                df_prog.loc[prog_idx, "count"] = len(prog_tick_prog)
                df_prog.loc[prog_idx, "channel"] = prog_tick_chan
                df_prog.loc[prog_idx, "program"] = prog_tick_prog
                
                prog_tick_chan, prog_tick_prog = initialize_prog_ticks()
        
            prog_tick_chan.append(msg.channel)
            prog_tick_prog.append(msg.program)
            
        
    print(prog_cur_time)
    display(df_note)
    print('---------------------')
    display(df_ctl)
    print('---------------------')
    display(df_prog)
    print("==========================")
                        


Track 1: He's A Pirate
Track 2: Right Hand
0


,tick,msg_velocity,count,msg_type,msg_channel,msg_note
1,160,[110],1,[note_on],[0],[62]
2,192,[0],1,[note_off],[0],[62]
3,286,[110],1,[note_on],[0],[62]
4,288,[0],1,[note_off],[0],[62]
5,448,[110],1,[note_on],[0],[62]
...,...,...,...,...,...,...
462,41998,"[0, 0]",2,"[note_off, note_off]","[0, 0]","[69, 79]"
463,42158,"[110, 110]",2,"[note_on, note_on]","[0, 0]","[69, 76]"
464,42414,"[0, 0, 110, 110, 110]",5,"[note_off, note_off, note_on, note_on, note_on]","[0, 0, 0, 0, 0]","[69, 76, 65, 69, 74]"
465,42734,"[0, 0, 0]",3,"[note_off, note_off, note_off]","[0, 0, 0]","[65, 69, 74]"


---------------------


,tick,count,control,value,channel
1,2,6,"[64, 7, 10, 7, 10, 64]","[0, 127, 64, 127, 64, 127]","[0, 0, 0, 0, 0, 0]"
2,4,2,"[64, 64]","[0, 127]","[0, 0]"
3,6,2,"[64, 64]","[0, 127]","[0, 0]"
4,8,2,"[64, 64]","[0, 127]","[0, 0]"
5,10,2,"[64, 64]","[0, 127]","[0, 0]"
...,...,...,...,...,...
76,4976,2,"[64, 64]","[0, 127]","[0, 0]"
77,5362,2,"[64, 64]","[0, 127]","[0, 0]"
78,5778,2,"[64, 64]","[0, 127]","[0, 0]"
79,6194,2,"[64, 64]","[0, 127]","[0, 0]"


---------------------


...

,tick,msg_velocity,count,msg_type,msg_channel,msg_note
1,528,"[110, 110]",2,"[note_on, note_on]","[1, 1]","[26, 38]"
2,576,"[0, 0]",2,"[note_off, note_off]","[1, 1]","[26, 38]"
3,832,"[110, 110]",2,"[note_on, note_on]","[1, 1]","[26, 38]"
4,864,"[0, 0]",2,"[note_off, note_off]","[1, 1]","[26, 38]"
5,1120,"[110, 110]",2,"[note_on, note_on]","[1, 1]","[26, 38]"
...,...,...,...,...,...,...
618,46196,"[0, 0]",2,"[note_off, note_off]","[1, 1]","[38, 50]"
619,46290,"[110, 110]",2,"[note_on, note_on]","[1, 1]","[38, 50]"
620,46292,"[0, 0]",2,"[note_off, note_off]","[1, 1]","[38, 50]"
621,46386,"[110, 110]",2,"[note_on, note_on]","[1, 1]","[38, 50]"


---------------------


,tick,count,control,value,channel
1,2304,4,"[7, 10, 7, 10]","[127, 64, 127, 64]","[1, 1, 1, 1]"
2,2336,2,"[64, 64]","[0, 127]","[1, 1]"
3,2338,2,"[64, 64]","[0, 127]","[1, 1]"
4,2340,2,"[64, 64]","[0, 127]","[1, 1]"
5,2342,2,"[64, 64]","[0, 127]","[1, 1]"
...,...,...,...,...,...
72,2566,2,"[64, 64]","[0, 127]","[1, 1]"
73,2568,2,"[64, 64]","[0, 127]","[1, 1]"
74,2570,2,"[64, 64]","[0, 127]","[1, 1]"
75,2572,2,"[64, 64]","[0, 127]","[1, 1]"


---------------------


,program,tick,count,channel
1,[],0,0,[]
2,[0],0,1,[1]


In [32]:
track_dict = {}
track_dict['MetaMessage'] = []
track_dict['MetaMessage'].append(345)

print(track_dict)

{'MetaMessage': [345]}


In [18]:
for idx, track in enumerate(mid.tracks):
    print(f"Track {i + 1}: {track.name}")
    
    for m

Track 1: He's A Pirate
Track 2: Right Hand
Track 3: Left Hand


In [9]:
mid.tracks[0]

MidiTrack([
  MetaMessage('track_name', name="He's A Pirate", time=0),
  MetaMessage('text', text='By Written by Klaus Badelt', time=0),
  MetaMessage('copyright', text='Arrangement © 2004 Cerullean Pictures ', time=0),
  MetaMessage('copyright', text='Original Score © Walt Disney Pictures', time=0),
  MetaMessage('text', text='Generated by NoteWorthy Composer', time=0),
  MetaMessage('set_tempo', tempo=300000, time=0),
  MetaMessage('key_signature', key='F', time=0),
  MetaMessage('time_signature', numerator=6, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
  MetaMessage('end_of_track', time=0)])

In [73]:
len(mid.tracks[1])

1285

In [14]:
file_name2 = "jesu1.mid"
file_path2 = os.path.join("midi_data", file_name2)

mid2 = MidiFile(file_path2)
print(mid2)

MidiFile(type=1, ticks_per_beat=120, tracks=[
  MidiTrack([
    MetaMessage('sequencer_specific', data=(0, 0, 65), time=0),
    MetaMessage('track_name', name='JESU        ', time=0),
    MetaMessage('key_signature', key='G', time=0),
    MetaMessage('midi_port', port=0, time=0),
    MetaMessage('time_signature', numerator=9, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('set_tempo', tempo=500000, time=38340),
    MetaMessage('end_of_track', time=0)]),
  MidiTrack([
    MetaMessage('track_name', name="Jesu, Joy of Man's", time=0),
    MetaMessage('instrument_name', name='Synths Strings 1', time=0),
    MetaMessage('midi_port', port=0, time=0),
    Message('control_change', channel=0, control=0, value=80, time=0),
    Message('control_change', channel=0, control=32, value=0, time=0),
    Message('program_change', channel=0, program=50, time=0),
    Message('note_on', channel=0, note=67, velocity=104, time=60),
    Message('note_on', channel=

In [15]:
len(mid2.tracks)

8

In [16]:
mid2.tracks[0]

MidiTrack([
  MetaMessage('sequencer_specific', data=(0, 0, 65), time=0),
  MetaMessage('track_name', name='JESU        ', time=0),
  MetaMessage('key_signature', key='G', time=0),
  MetaMessage('midi_port', port=0, time=0),
  MetaMessage('time_signature', numerator=9, denominator=8, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
  MetaMessage('set_tempo', tempo=500000, time=38340),
  MetaMessage('end_of_track', time=0)])

In [17]:
mid2.tracks[1]

MidiTrack([
  MetaMessage('track_name', name="Jesu, Joy of Man's", time=0),
  MetaMessage('instrument_name', name='Synths Strings 1', time=0),
  MetaMessage('midi_port', port=0, time=0),
  Message('control_change', channel=0, control=0, value=80, time=0),
  Message('control_change', channel=0, control=32, value=0, time=0),
  Message('program_change', channel=0, program=50, time=0),
  Message('note_on', channel=0, note=67, velocity=104, time=60),
  Message('note_on', channel=0, note=67, velocity=0, time=48),
  Message('note_on', channel=0, note=69, velocity=104, time=12),
  Message('note_on', channel=0, note=69, velocity=0, time=48),
  Message('note_on', channel=0, note=71, velocity=104, time=12),
  Message('note_on', channel=0, note=71, velocity=0, time=48),
  Message('note_on', channel=0, note=74, velocity=104, time=12),
  Message('note_on', channel=0, note=74, velocity=0, time=48),
  Message('note_on', channel=0, note=72, velocity=104, time=12),
  Message('note_on', channel=0, note=7

In [ ]:
// pending